In [1]:
import pandas as pd
import time

In [2]:
path = '/Users/wanghan/Projects/MacMic/data/Messaging.csv'
message = pd.read_csv(path)

/Users/wanghan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
message = message.dropna()

In [33]:
message = message.reset_index(drop=True)

In [14]:
#Loading gensim and nltk libraries
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
# np.random.seed(2018)
import nltk
nltk.download('wordnet')

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
[nltk_data] Downloading package wordnet to /Users/wanghan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [38]:
from nltk.stem.porter import *
def lemmatize_stemming(text):
    stemmer = PorterStemmer()
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2:
            result.append(lemmatize_stemming(token))
    return result


In [39]:
from gensim import corpora, models
def generate_dictionary_corpus(chat):
    processed_docs = chat['CHAT Messages Message Body'].map(preprocess)
    dictionary = gensim.corpora.Dictionary(processed_docs)
    dictionary.filter_extremes(no_below=15, no_above=0.8)
    bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
    return(processed_docs, dictionary, bow_corpus)

def generate_tfidf(bow_corpus):
    tfidf = models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf[bow_corpus]
    return(corpus_tfidf)

In [40]:
documents = message[['CHAT Messages Message Body']]

In [41]:
processed_docs, dictionary, bow_corpus = generate_dictionary_corpus(documents)
corpus_tfidf = generate_tfidf(bow_corpus)

In [50]:
message['clean_text'] = processed_docs

In [51]:
message.to_csv('/Users/wanghan/Projects/MacMic/data/Messaging.csv')

,Users User ID,CHATting Stylists Stylist ID,CHAT Messages Created Time,CHAT Messages Sender,CHAT Messages Message Body,clean_text
0,216703.0,98569.0,6/12/2019 20:21,stylist,Sounds good 😊,"[sound, good]"
1,313354.0,281470.0,6/12/2019 20:21,customer,Sorry that was ment to say dollars,"[sorri, ment, dollar]"
2,202942.0,156257.0,6/12/2019 20:21,stylist,Great - would you like a box for each of the b...,"[great, like, box, boy]"
3,313354.0,281470.0,6/12/2019 20:20,customer,So let's say one item cost 20 and 15 and if sh...,"[let, item, cost, keep, item, pay, dolla]"
4,313354.0,281470.0,6/12/2019 20:19,stylist,"Yes, you only pay for the items you decide to ...","[ye, pay, item, decid, send, iphon]"
